<a href="https://colab.research.google.com/github/ViekyKayal/Llama-2-model/blob/main/LlamaModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -r /content/drive/MyDrive/chatbot/llama/req.txt

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA

In [ ]:
DB_FAISS_PATH = '/content/drive/MyDrive/chatbot/llama/data/db_faiss'

custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:

def set_custom_prompt():
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

#Retrieval QA Chain
def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 2}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )
    return qa_chain

In [ ]:
#Loading the model
def load_llm():
    # Load the locally downloaded model here
    llm = CTransformers(
        model = "TheBloke/Llama-2-7B-Chat-GGML",
        model_type="llama",
        max_new_tokens = 512,
        temperature = 0.5
    )
    return llm

In [ ]:
#QA Model Function
def qa_bot():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = FAISS.load_local(DB_FAISS_PATH, embeddings)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)

    return qa

In [ ]:
#output function
def final_result(query):
    qa_result = qa_bot()
    response = qa_result({'query': query})
    return response


In [ ]:
query="who is arka"
cb=final_result(query)
print(cb)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

{'query': 'who is arka', 'result': 'Arka is a young boy living in Willowbrook.', 'source_documents': [Document(page_content="in Willowbrook, known far and wide as the adventurous boy wh o had truly lived his dreams.  \nAnd so, Arka's story became a legend in its own right, inspiring generations of children to dream big \nand embark on their own daring adventures, for they knew that the world was full of magic and \nwonder, waiting to be discover ed.", metadata={'source': '/content/drive/MyDrive/chatbot/llama/data/story.pdf', 'page': 0}), Document(page_content='Title: "Arka\'s Daring Adventure"  \nOnce upon a time in the quaint village of Willowbrook, there lived a curious and adventurous young \nboy named Arka. Arka had always been fascinated by tales of far -off lands, hidden treasures, and \nmythical creatures. He spent his days reading books about da ring explorers and their incredible \nadventures. Little did he know that his own adventure was about to begin.', metadata={'source': 

In [ ]:
print("user:",cb['query'])
print("chatbot:",cb['result'])

user: who is arka
chatbot: Arka is a young boy living in Willowbrook.


In [ ]:
while True:
  query=input("user:")
  if query=="exit": break
  cb=final_result(query)
  print("chatbot:",cb['result'])

user:who is arka


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

chatbot: Arka is a young boy from Willowbrook who is known for being adventurous and reading books about daring explorers.
user:who is his friend


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

chatbot: Arka's friend is Polly.
user:exit


In [ ]:
for i in cb:
  print(i,":",cb[i])
for i in cb["source_documents"]:
  print(i)

query : who is his friend
result : Arka's friend is Polly.
source_documents : [Document(page_content='made his final wish to return ho me, promising to visit the Isle of Enchantment whenever he could.  \nWith a heavy heart, he and Polly bid farewell to the magical island, but they knew that their \nadventures would continue in Willowbrook and beyond.  \nArka returned to his village, where he shared his incredible journey with everyone. The lessons he \nhad learned and the courage he had gained from the Isle of Enchantment made him a beloved figure', metadata={'source': '/content/drive/MyDrive/chatbot/llama/data/story.pdf', 'page': 0}), Document(page_content="in Willowbrook, known far and wide as the adventurous boy wh o had truly lived his dreams.  \nAnd so, Arka's story became a legend in its own right, inspiring generations of children to dream big \nand embark on their own daring adventures, for they knew that the world was full of magic and \nwonder, waiting to be discover ed.", me

In [ ]:
for i in cb["source_documents"]:
  print(str(i))

page_content='made his final wish to return ho me, promising to visit the Isle of Enchantment whenever he could.  \nWith a heavy heart, he and Polly bid farewell to the magical island, but they knew that their \nadventures would continue in Willowbrook and beyond.  \nArka returned to his village, where he shared his incredible journey with everyone. The lessons he \nhad learned and the courage he had gained from the Isle of Enchantment made him a beloved figure' metadata={'source': '/content/drive/MyDrive/chatbot/llama/data/story.pdf', 'page': 0}
page_content="in Willowbrook, known far and wide as the adventurous boy wh o had truly lived his dreams.  \nAnd so, Arka's story became a legend in its own right, inspiring generations of children to dream big \nand embark on their own daring adventures, for they knew that the world was full of magic and \nwonder, waiting to be discover ed." metadata={'source': '/content/drive/MyDrive/chatbot/llama/data/story.pdf', 'page': 0}
